In [1]:
from math import comb

import pandas as pd
import numpy as np
import numpy.polynomial.polynomial as poly
import matplotlib.pyplot as plt
from scipy.stats import norm

/home/egor/anaconda3/envs/lesta/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Краткие ответы

## Задача 1 - Работа с распределением 
1. Среднее: 31.75, медиана: 30.68
2. 77% игроков
3. 61 бой
4. 68 боёв
5. 47 боёв

## Задача 2 - Контейнеры
1. Рассуждения и попытка аналитического решения. Ответ не нашел
2. Аналитическое решение:

    Вероятность получения одного героя:
$$
        P(1) = C^1_{15}p_h(1- p_h)^{14}
$$
    Вероятность получения двух героев:
$$
        P(2) = C^2_{15}p_h^2(1- p_h)^{13}
$$
    Вероятность получения трех героев:
$$
        P(3) = C^3_{15}p_h^3(1- p_h)^{12}
$$
    Вероятность получения четырех героев:
$$
        P(4) = C^4_{15}p_h^4(1- p_h)^{11}
$$
    Вероятность, что ни одного героя не будет получено
$$
    P(0) = C^0_{15}(1- p_h)^{15} = (1- p_h)^{15}
$$

# Задача 1 

Дано распределение игроков некоторой игры по количеству боев за месяц (учитываются только игроки, проведшие хотя бы 1 бой за месяц, первый столбец, например, показывает, что 544 игрока сыграли за месяц от 1 до 3 боев, считаем, что игроков, проведших более 100 боев не было)

## 1.1 Оцените среднее число боев, приходящееся на одного игрока в месяц и медиану числа боев, приходящегося на одного игрока

### Решение

Задача сводится к оценке среднего значения и медианы группового распределения частот.

1. Среднее значение считается по формуле:
$$
    Mean = \frac{\sum f \cdot m}{\sum f} \mbox{, где} \\
    \mbox{m - это середины заданных интервалов и} \\
    \mbox{f - частота (кол-во игроков) на интервале}
$$
    
    Значения $f$ - известны, значения $m$ считаются, как среднее нижней и верхней границы интервала:
$$
    m = \frac{a + b}{2}\mbox{, где}\\
    \mbox{a и b - нижняя и верхняя границы интервала соответственно}
$$

    Код ниже считает $m$ для каждого интервала

In [2]:
# Начальные условия
solution_dict = {"Число боев": ["1-3", "4-10", 
                                "11-20", "21-30", 
                                "31-40", "41-50",
                                "51-75", "76-100"],
                "Число игроков, f": [533, 689, 
                                  1165, 1354, 
                                  1104, 921, 
                                  537, 475]}

# Добавим m
ranges = [(1, 3), (4, 10),     # интервалы в численном виде
          (11, 20), (21, 30), 
          (31, 40), (41, 50), 
          (51, 75), (76, 100)] 
m_values = [(a + b) / 2 for a, b in ranges]
solution_dict["Середина интервала, m"] = m_values

# Выводим результаты в таблцу
pd.DataFrame(solution_dict)

,Число боев,"Число игроков, f","Середина интервала, m"
0,1-3,533,2.0
1,4-10,689,7.0
2,11-20,1165,15.5
3,21-30,1354,25.5
4,31-40,1104,35.5
5,41-50,921,45.5
6,51-75,537,63.0
7,76-100,475,88.0


   **Теперь можно посчитать среднее**

In [3]:
f = solution_dict["Число игроков, f"]
m = solution_dict["Середина интервала, m"]
cross_list = list(zip(f,m))
products = [f_i * m_i for f_i, m_i in cross_list]
solution_dict["f * m"] = products
sum_of_products = sum(products)
mean =  sum_of_products / sum(f)
print(f"Среднее число боев на одного игрока: {mean}")
pd.DataFrame(solution_dict)

Среднее число боев на одного игрока: 31.75007376807318


,Число боев,"Число игроков, f","Середина интервала, m",f * m
0,1-3,533,2.0,1066.0
1,4-10,689,7.0,4823.0
2,11-20,1165,15.5,18057.5
3,21-30,1354,25.5,34527.0
4,31-40,1104,35.5,39192.0
5,41-50,921,45.5,41905.5
6,51-75,537,63.0,33831.0
7,76-100,475,88.0,41800.0


2. Чтобы найти медиану числа боев, сначала надо найти интервал, в котором заключена медиана. Необходимо айти в выборке такой интервал числа боев, для которго все интервалы до будут включать в себя половину игроков, которые сыграли меньше боев, а интервалы после - половину, которая сыграла больше. 

    Вычислим половину от общего числа игроков выборке:

In [4]:
players_sum = sum(f)
players_half = players_sum / 2
print(players_half)

3389.0


    Составим таблицу порядковых номеров игроков, которые содержит каждый интервал, указывая номер последнего игрока в интервале (кумулятивные частоты интервала):

In [5]:
players_num = [f[0]]
for freq in f[1:]:
    players_num.append(freq + players_num[-1])
solution_dict["Содержит игроков до №"] = players_num
pd.DataFrame(solution_dict)

,Число боев,"Число игроков, f","Середина интервала, m",f * m,Содержит игроков до №
0,1-3,533,2.0,1066.0,533
1,4-10,689,7.0,4823.0,1222
2,11-20,1165,15.5,18057.5,2387
3,21-30,1354,25.5,34527.0,3741
4,31-40,1104,35.5,39192.0,4845
5,41-50,921,45.5,41905.5,5766
6,51-75,537,63.0,33831.0,6303
7,76-100,475,88.0,41800.0,6778


    В интервале 31-40 лежат игроки с номерами 3742 - 4845. Там же лежит и половина от общего числа игроков: 3389. Теперь из найденного интервала можно оценить медиану. 
    
    Медиана оценивается по формуле:
$$
    Median = a + \frac{(n/2) - B}{G}\mbox{, где}\\
    \mbox{n - общее число игроков}\\
    \mbox{B - кумулитивная частота интервала, предшествующего найденому}\\
    \mbox{G - частота найденного интервала}\\
    \mbox{W - ширина группы, (b - a)}\\
$$

**Оценка медианы:**

In [6]:
median_idx = 4
B = solution_dict["Содержит игроков до №"][median_idx - 1]
G = f[median_idx]
a, b = ranges[median_idx]
W = b - a

median = a + ((players_sum / 2) - B) / G
median

30.681159420289855

## 1.2 Оцените, какая доля игроков провела 15 и более боев за месяц

### Решение

1. Для оценки допустим, что частоты число игроков распределено по нормальному закону. Чтобы дать оцнку доли игроков, необходимо задать параметры распределения $a$ и $\sigma$. 

    Параметр $a$ - это среднее число боев, приходящееся на одного игрока. Из предыдущего задания мы знаем, что $a = 31.75$
    
    $\sigma$ - среднеквадратичное отклонение, которе для данного группового распределения частот будет считаться по формуле:
    
$$
    \sigma = \sqrt{\frac{\sum m^2 f}{n} - \left(\frac{\sum fm}{n}\right)^2}
$$

    Все необходимые значения уже были посчитаны в предыдущем задании, осталось их только подставить в формулу:

In [7]:
products_squares = [m * m * f for f, m in cross_list]
mean_squares = sum(products_squares) / players_sum
mean_squared = (sum_of_products / players_sum)**2
sigma = np.sqrt(mean_squares - mean_squared)
sigma

22.630600687641792

2. Используем модель нормального распределения из scipy.stats и подставим в неё параметры $a$ и $\sigma$. Затем для полученной модели **посчитаем долю игроков, которая провела 15 и более боев за месяц**

In [8]:
target_num = 15

distribution = norm(mean, sigma)
less_then_target = distribution.cdf(target_num)
more_then_target = 1 - less_then_target
more_then_target

0.7703959734295673

### 1.3 Оцените, какое число боев может быть достаточным для акции?

**Описание акции:** Допустим, мы хотим сделать в игре событие длиной в месяц на получение ценной награды за проведенные бои и хотим сделать так, чтобы эту награду получили не более 10% наиболее активных (по числу боев) игроков. 

**Примечание:** При оценке стоит учесть, что значительно сокращать число игроков, прошедших событие полностью относительно целевых 10% считаем нежелательным.

### Решение

Задача обратная предыдущей - нам известно доля игроков, и теперь мы ищем число боев. Для решения используется обратная функция стандартного нормального распределения (преобразовывающая нормальную вероятность р в нормальное значение z)

In [9]:
target_fraction = 0.1
reverse_fraction = 1 - target_fraction
target_num_of_games = distribution.ppf(reverse_fraction)
target_num_of_games

60.75235550853523

Т.к. по условию событие должны проийти *не более* 10% игроков, округляем посчитанное значение в большую сторону. **Для получения награды игрок должен сыграть 61+ бой за месяц.**

## 1.4 . Как изменится ответ на задание 1.3 с учетом допущения?
**Допущение:** Предположим, что награда достаточно ценна для того, чтобы игроки увеличили свое число боев в месяц на 20%. 

### Решение

Внесём изменения в нашу предыдущую модель. Увеличения число боев в месяц на 20% приведет к смещению среднего рампределения на те же 20%. Дисперсия/среднеквадратичное отклонение при этом остаётся неизменным

In [10]:
inc = 0.2
mean_hypo = (1 + inc)* mean
distribution_hypo = norm(mean_hypo, sigma)
target_num_of_games_hypo = distribution_hypo.ppf(reverse_fraction)
target_num_of_games_hypo

67.10237026214986

Как и в прошлый раз, округляем в большую торону: **68+ боёв должен сыграть игрок для получения награды**.

## 1.5 Если событие из пункта 1.3 будет длиться 2 недели, а не месяц, как может измениться требуемое число боев?

## Решение

**Допущение:** сокращение длительности до 2-х недель равносильно, сокращению длительности события в 2 раза (месяц - 28 дней) 

Предположим, что, уменьшив длительность события, уменьшается и разброс распределения игроков по группам числа боёв. Иными словами: среднеквадратическое отклонения уменьшилось в 2 раза (среднее остается тем же).  

Решим задачу 1.3 для модели с новыми параметрами:

In [11]:
new_sigma = sigma / 2
new_distribution = norm(mean, new_sigma)
new_target_num_of_games = new_distribution.ppf(reverse_fraction)
new_target_num_of_games

46.251214638304205

Округляем в большую сторону: **47+ боёв должен сыграть игрок для получения награды**.

# Задача 2

## 2.1 Чему должна быть равна вероятность получения категории героев?
**Условие:** Cредняя цена получения героя через контейнеры равна цене прямой покупки героя

### Решение (аналитиеское, неполное)
Представим, что каждый раз, покупая котнейнер, игрок приобретает *часть* целевого героя $A$. Тогда, взяв в расчёт стоимости покупки героя $h = 5500$ и контейнера $c = 1000$, мы можем расчитать желаемую вероятность выпадения именно этого героя, как отношение:

$$
    P(A) = \frac{c}{h} = \frac{1000}{5500} = \frac{2}{11}
$$

Это желаемая вероятность выпадения героя $A$. Другой способ вычислить эту же вероятность - рассмотреть её, как сумму вероятностей 3-х сценариев выпадения категории героя по биноминальному закону с параметрами $n=\mbox{от 2 до 4}$, $m=11$, $p=x$, $q=1-x$ с последующим вытаскиванием героя $A$. Формулы вероятностей для каждого из сценариев:
1. 2 выпадения категории героев, вытащил: $AA$:
$$
    P_1(A) = C^2_{11}x^2q^9\cdot\frac{1}{4}\cdot\frac{1}{3}
$$
2. 3 выпадения категории героев, вытащил: $\bar{A}AA$:
$$
   P_2(A) = C^3_{11}x^3q^8\cdot\frac{3}{4}\cdot\frac{1}{3}\cdot\frac{1}{2}
$$
2. 4 выпадения категории героев, вытащил: $\bar{A}\bar{A}AA$:
$$
   P_3(A) = C^4_{11}x^4q^7\cdot\frac{3}{4}\cdot\frac{2}{3}\cdot\frac{1}{2}\cdot1
$$

Сумма этих вероятностей даёт вероятность выпадения героя $A$. Запишем эту сумму, выразив q через x и подставив значение $P(A)$:
$$
    P(A) = P_1(A) + P_2(A) + P_3(A) =  C^2_{11}x^2(1 - x)^9\cdot\frac{1}{4}\cdot\frac{1}{3} + 
    C^3_{11}x^3(1- x)^8\cdot\frac{3}{4}\cdot\frac{1}{3}\cdot\frac{1}{2} + 
    C^4_{11}x^4(1- x)^7\cdot\frac{3}{4}\cdot\frac{2}{3}\cdot\frac{1}{2}\cdot1 = \frac{2}{11}
$$

Если в полученном уравнении раскрыть все скобки и перенести свободный коэффициент через знак равенства, то мы получим полином 13 степени, корни которого можно попытаться вычислить, используя численные методы. 

Я попробовал так сделать - у меня не получилось, не нашёл реальных корней (если интересно, см. код ниже).

***Если у этой задачи есть другое/правильное решение, то мне оно пока неизвестно. Но я пытался :)***

In [14]:
# Численые коэффициенты из формул выше
A = comb(11, 2) / 4 / 3
B = comb(11, 3) * 3 / 4 / 3 / 2
C = comb(11, 4) * 3 / 4  * 2 / 3 / 2

coefficients = [2 / 11, 
                0, 
                A, 
                -9 * A + B, 
                36 * A - 10 * B + C,
                -84 * A + 45 * B + 7 * C,
                126 * A - 120 * B + 21 * C,
                -126 * A + 210 * B - 35 * C,
                84 * A - 252 * B + 35 * C,
                -36 * A + 175 * B - 21 * C,
                2 * A - 36 * B + 7 * C,
                - A + 45 * B - C,
                - 3 * B,
                B]
coefficients

[0.18181818181818182,
 0,
 4.583333333333333,
 -20.625,
 41.25,
 1120.625,
 -165.0,
 866.25,
 -1925.0,
 1711.875,
 -155.83333333333337,
 841.0416666666666,
 -61.875,
 20.625]

In [15]:
roots = poly.polyroots(coefficients)
roots

array([-0.4712443 -1.51399778j, -0.4712443 +1.51399778j,
       -0.36385005-0.67526086j, -0.36385005+0.67526086j,
       -0.22972781+0.j        , -0.0434869 -0.13533817j,
       -0.0434869 +0.13533817j,  0.14383191-0.11548524j,
        0.14383191+0.11548524j,  0.84224785-0.5105296j ,
        0.84224785+0.5105296j ,  1.50736539-6.05163396j,
        1.50736539+6.05163396j])

## 2.2 Рассчитайте вероятности получения одного, двух, трех, всех четырех героев, а также того, что ни одного героя не будет получено при покупке 15 контейнеров.

### Решение (аналитическое)

Допустим в предыдущем задании я вычислил оптимальную вероятность получения категории героев, и она равна $p_h$. Т.к. теперь нам не важно какой герой вытаскивается, можно посчитать все вероятности по биноминалньому закону с параметрами $m = 15$ $n = \mbox{от 0 до 4}$, $p = p_h$ и $q = 1 - p_h$:
1. Вероятности получения одного героя:
$$
    P(1) = C^1_{15}p_h(1- p_h)^{14}
$$
2. Вероятности получения двух героев:
$$
    P(2) = C^2_{15}p_h^2(1- p_h)^{13}
$$
3. Вероятности получения трех героев:
$$
    P(3) = C^3_{15}p_h^3(1- p_h)^{12}
$$
3. Вероятности получения четырех героев:
$$
    P(4) = C^4_{15}p_h^4(1- p_h)^{11}
$$
4. Вероятность, что ни одного героя не будет получено
$$
    P(0) = C^0_{15}(1- p_h)^{15} = (1- p_h)^{15}
$$